# **Install the libraries**

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# **Create a session**

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType

In [ ]:
spark_session = SparkSession \
  .builder \
  .appName("spark_reading_streaming") \
  .master("yarn").getOrCreate()

In [7]:
project_number = 823002731253
location = "us-central1-a"
topic_id = "log-messages-topic"
subscription_id = "log-messages-subscription"

# **Reading From Pub/Sub - Spark Streaming**

In [ ]:
dataframe = (
    spark_session.readStream.format("pubsublite")
    .option(
        "pubsublite.subscription",
        f"projects/{project_number}/locations/{location}/subscriptions/{subscription_id}",
    )
    .load()
)

In [ ]:
dataframe = dataframe.withColumn("data", dataframe.data.cast(StringType()))

query = (
    dataframe.writeStream.format("console")
    .outputMode("append")
    .trigger(processingTime="1 second")
    .start()
)

query.awaitTermination(120)
query.stop()